<a href="https://colab.research.google.com/github/AnshikaMishra23/Medical_Image_to_report/blob/main/Medical_Image_to_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install opencv-python matplotlib tensorflow

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [9]:
import zipfile
zip_ref = zipfile.ZipFile('cancer-cells.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [11]:
!ls /content


basophil	  erythroblast	myeloblast   seg_neutrophil
cancer-cells.zip  monocyte	sample_data


In [12]:
import cv2
import os
import numpy as np

data = []
labels = []

# Set the correct base path and folder names
base_path = "/content"
class_names = ['basophil', 'erythroblast', 'monocyte', 'myeloblast', 'seg_neutrophil']

# Loop through each folder and load images
for i, cls in enumerate(class_names):
    folder = os.path.join(base_path, cls)
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        try:
            img = cv2.imread(img_path)
            img = cv2.resize(img, (64, 64))
            data.append(img)
            labels.append(i)
        except:
            continue

# Convert to numpy arrays
X = np.array(data) / 255.0
y = np.array(labels)

print("✅ Data Loaded:")
print("X shape:", X.shape)
print("y shape:", y.shape)


✅ Data Loaded:
X shape: (5000, 64, 64, 3)
y shape: (5000,)


In [13]:
import os
import cv2
import numpy as np

data = []
labels = []

base_path = "/content"
class_names = ['basophil', 'erythroblast', 'monocyte', 'myeloblast', 'seg_neutrophil']

for i, cls in enumerate(class_names):
    folder = os.path.join(base_path, cls)
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        try:
            img = cv2.imread(img_path)
            img = cv2.resize(img, (64, 64))
            data.append(img)
            labels.append(i)
        except:
            continue

X = np.array(data) / 255.0
y = np.array(labels)


In [14]:
print("Dataset shape:", X.shape)
print("Labels shape:", y.shape)
print("Example label:", y[0])


Dataset shape: (5000, 64, 64, 3)
Labels shape: (5000,)
Example label: 0


In [15]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# One-hot encode the labels
y_cat = to_categorical(y, num_classes=len(set(y)))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Train shape: (4000, 64, 64, 3)
Test shape: (1000, 64, 64, 3)


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(5, activation='softmax')  # 5 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,625,797 (6.20 MB)

 Trainable params: 1,625,797 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 26s 192ms/step - accuracy: 0.5496 - loss: 1.0938 - val_accuracy: 0.8730 - val_loss: 0.3499
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 198ms/step - accuracy: 0.8628 - loss: 0.3810 - val_accuracy: 0.9330 - val_loss: 0.2129
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 44s 217ms/step - accuracy: 0.9250 - loss: 0.2437 - val_accuracy: 0.9580 - val_loss: 0.1266
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 36s 176ms/step - accuracy: 0.9398 - loss: 0.1990 - val_accuracy: 0.9590 - val_loss: 0.1230
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 44s 199ms/step - accuracy: 0.9343 - loss: 0.1785 - val_accuracy: 0.9680 - val_loss: 0.1031
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 194ms/step - accuracy: 0.9590 - loss: 0.1305 - val_accuracy: 0.9740 - val_loss: 0.1019
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 39s 179ms/step - accuracy: 0.9615 - loss: 0.1213 - val_accuracy: 0.9630 - val_loss: 0.1029
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 24s 195ms/step - accuracy: 0.9638 - loss: 0

In [18]:
import numpy as np

class_names = ['basophil', 'erythroblast', 'monocyte', 'myeloblast', 'seg_neutrophil']

def generate_report(pred_class):
    diagnosis = class_names[pred_class]
    if diagnosis in ['myeloblast', 'erythroblast']:
        condition = "⚠️ Potential cancerous or immature cells detected."
        advice = "Consult a hematologist. Further biopsy or blood tests may be needed."
    else:
        condition = "✅ Normal or mature blood cell type detected."
        advice = "No urgent issues, but routine check-up is advised."

    return f"""🩸 **Cell Type**: {diagnosis}
🧠 **Diagnosis**: {condition}
💊 **Advice**: {advice}
"""

# Predict on a test sample
pred = model.predict(np.expand_dims(X_test[0], axis=0))
pred_class = np.argmax(pred)
print(generate_report(pred_class))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
🩸 **Cell Type**: erythroblast
🧠 **Diagnosis**: ⚠️ Potential cancerous or immature cells detected.
💊 **Advice**: Consult a hematologist. Further biopsy or blood tests may be needed.



In [19]:
import gradio as gr
import cv2
import numpy as np

# Your class names
class_names = ['basophil', 'erythroblast', 'monocyte', 'myeloblast', 'seg_neutrophil']

# Report generator
def generate_report(pred_class):
    diagnosis = class_names[pred_class]
    if diagnosis in ['myeloblast', 'erythroblast']:
        condition = "⚠️ Potential cancerous or immature cells detected."
        advice = "Consult a hematologist. Further biopsy or blood tests may be needed."
    else:
        condition = "✅ Normal or mature blood cell type detected."
        advice = "No urgent issues, but routine check-up is advised."

    return f"""🩸 **Cell Type**: {diagnosis}
🧠 **Diagnosis**: {condition}
💊 **Advice**: {advice}"""

# Gradio-compatible function
def classify_and_report(image):
    image = cv2.resize(image, (64, 64))
    image = np.expand_dims(image, axis=0) / 255.0
    pred = model.predict(image)
    pred_class = np.argmax(pred)
    report = generate_report(pred_class)
    return report


In [20]:
interface = gr.Interface(
    fn=classify_and_report,
    inputs=gr.Image(type="numpy", label="Upload Blood Cell Image"),
    outputs=gr.Textbox(label="Medical Report"),
    title="🧬 Blood Cell Type Detector",
    description="Upload a microscope image of a blood cell to detect its type and get a report."
)

interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://38dff40288b75791fd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
